# Textklassifikation

Die Klassifikation von Texten ist einer der zentralen Anwendungsfälle von NLP. 
Er begegnet uns ständig, etwa in Form eines Spam-Filters in unserem Email-Client oder bei der Intent-Erkennung bei einem Sprachassistenten.

Gleichzeitig eignet sich die Textklassifikation als Beispiel dafür, welche Fortschritte in den letzten Jahren im Bereich NLP gemacht wurden,
etwa bei der Vektorisierung mit *Word2Vec* oder dem Einsatz von transformer-basierten Modellen wie *BERT*.

Wir werden als Beispiel die Daten aus dem Wettbewerb *GermEval 2018* (s. `../data/GermEval-2018`) verwenden.

## Aufgabe 1: Lesen & Aufbereiten der Texte

Um die Texte zu klassifizieren, müssen wir die Trainingsdaten lesen und aufbereiten. Schauen Sie sich dazu zunächst die ersten 10 Trainingsdatensätze an (s. folgende Zelle).

In [3]:
!head -10 ../data/GermEval-2018/germeval2018.training.txt

@corinnamilborn Liebe Corinna, wir würden dich gerne als Moderatorin für uns gewinnen! Wärst du begeisterbar?	OTHER	OTHER
@Martin28a Sie haben ja auch Recht. Unser Tweet war etwas missverständlich. Dass das BVerfG Sachleistungen nicht ausschließt, kritisieren wir.	OTHER	OTHER
@ahrens_theo fröhlicher gruß aus der schönsten stadt der welt theo ⚓️	OTHER	OTHER
@dushanwegner Amis hätten alles und jeden gewählt...nur Hillary wollten sie nicht und eine Fortsetzung von Obama-Politik erst recht nicht..!	OTHER	OTHER
@spdde kein verläßlicher Verhandlungspartner. Nachkarteln nach den Sondierzngsgesprächen - schickt diese Stümper #SPD in die Versenkung.	OFFENSE	INSULT
@Dirki_M Ja, aber wo widersprechen die Zahlen denn denen, die im von uns verlinkten Artikel stehen? In unserem Tweet geht es rein um subs. Geschützte. 2017 ist der gesamte Familiennachzug im Vergleich zu 2016 - die Zahlen, die Hr. Brandner bemüht - übrigens leicht rückläufig gewesen.	OTHER	OTHER
@milenahanm 33 bis 45 habe ich noch gar

### Aufgabe 1.1: Auffälligkeiten in den Texten

Was fällt Ihnen an den Texten auf? Welche Bestandteile sollten ggf. aus den Trainingsdaten entfernt werden, da sie das Training verfälschen würden?
Welche anderen Bereinigungen könnten sinnvoll sein?

> 💡 **Tipp:** 
> Fragen Sie gerne auch ChatGPT, was man bei der Bereinigung der Texte tun sollte. 

### Aufgabe 1.2: Lesen der Trainings- und Testdatensätze

Die Trainingsdaten liegen als Textdatei mit drei durch `TAB` getrennten Spalten vor.

| TEXT | COARSE | FINE |
| ---- | ------ | ---- |
| @corinnamilborn Liebe Corinna, wir würden dich gerne als Moderatorin für uns gewinnen! Wärst du begeisterbar?  | OTHER | OTHER |
| @Martin28a Sie haben ja auch Recht. Unser Tweet war etwas missverständlich. Dass das BVerfG Sachleistungen nicht ausschließt, kritisieren wir. | OTHER | OTHER |
| @ahrens_theo fröhlicher gruß aus der schönsten stadt der welt theo ⚓️ | OTHER | OTHER |
| @dushanwegner Amis hätten alles und jeden gewählt...nur Hillary wollten sie nicht und eine Fortsetzung von Obama-Politik erst recht nicht..! | OTHER | OTHER |
| @spdde kein verläßlicher Verhandlungspartner. Nachkarteln nach den Sondierzngsgesprächen - schickt diese Stümper #SPD in die Versenkung. | OFFENSE | INSULT | 

Lesen Sie die Daten und wandeln Sie ihn in eine Liste von `namedtuple` mit den Feldern `text`, `coarse_label` und `fine_label` um.

> 💡 **Tipp:** 
> ChatGPT kann Ihnen beim Code sicher helfen

In [2]:
import csv
from collections import namedtuple

# Schritt 2: Definieren des namedtuple
DataItem = namedtuple('DataItem', ['text', 'coarse_label', 'fine_label'])

# Erstellen einer leeren Liste, um die Datenitems zu speichern
data_items = []

# Schritt 3: Lesen der Datei
with open('../data/GermEval-2018/germeval2018.training.txt', 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter='\t')
    next(reader)  # Überspringen der Kopfzeile
    for row in reader:
        if row:  # Sicherstellen, dass die Zeile nicht leer ist
            item = DataItem(text=row[0], coarse_label=row[1], fine_label=row[2])
            data_items.append(item)

# Ausgeben der ersten 5 Datenelemente zur Überprüfung
for item in data_items[:5]:
    print(item)

DataItem(text='@Martin28a Sie haben ja auch Recht. Unser Tweet war etwas missverständlich. Dass das BVerfG Sachleistungen nicht ausschließt, kritisieren wir.', coarse_label='OTHER', fine_label='OTHER')
DataItem(text='@ahrens_theo fröhlicher gruß aus der schönsten stadt der welt theo ⚓️', coarse_label='OTHER', fine_label='OTHER')
DataItem(text='@dushanwegner Amis hätten alles und jeden gewählt...nur Hillary wollten sie nicht und eine Fortsetzung von Obama-Politik erst recht nicht..!', coarse_label='OTHER', fine_label='OTHER')
DataItem(text='@spdde kein verläßlicher Verhandlungspartner. Nachkarteln nach den Sondierzngsgesprächen - schickt diese Stümper #SPD in die Versenkung.', coarse_label='OFFENSE', fine_label='INSULT')
DataItem(text='@Dirki_M Ja, aber wo widersprechen die Zahlen denn denen, die im von uns verlinkten Artikel stehen? In unserem Tweet geht es rein um subs. Geschützte. 2017 ist der gesamte Familiennachzug im Vergleich zu 2016 - die Zahlen, die Hr. Brandner bemüht - übrige

### Aufgabe 1.3: Bereinigung der Trainingsdatensätze

Setzen Sie **mindestens** die folgenden Textbereinigungen um:

- Entfernen von *Twitter-Handles* wie `@corinnamilborn` (Warum ist das sinnvoll?)
- Entfernen des Hashtag-Zeichens `#` (Warum ist das sinnvoll?)

Zusätzlich können Sie mit anderen Bereinigungen experimentieren (Entfernung Groß- und Kleinschreibung, Stemming, ...). Im Deutschen bringt das allerdings nicht so viel.

In [3]:
import csv
import re
from collections import namedtuple

# Schritt 2: Definition des namedtuple
DataItem = namedtuple('DataItem', ['text', 'coarse_label', 'fine_label'])

data_items = []

def clean_text(text):
    text = re.sub(r'@\w+', '', text)  # Entfernen von Twitter-Handles
    text = re.sub(r'#', '', text)  # Entfernen von Hashtag-Zeichen
    # Hier können Sie weitere Bereinigungen hinzufügen
    text = text.lower()  # Optional: Umwandlung in Kleinbuchstaben
    return text.strip()

with open('../data/GermEval-2018/germeval2018.training.txt', 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter='\t')
    next(reader)  # Überspringen der Kopfzeile
    for row in reader:
        if row:  # Sicherstellen, dass die Zeile nicht leer ist
            cleaned_text = clean_text(row[0])
            item = DataItem(text=cleaned_text, coarse_label=row[1], fine_label=row[2])
            data_items.append(item)

# Die ersten 5 Datenelemente ausgeben
for item in data_items[:5]:
    print(item)

DataItem(text='sie haben ja auch recht. unser tweet war etwas missverständlich. dass das bverfg sachleistungen nicht ausschließt, kritisieren wir.', coarse_label='OTHER', fine_label='OTHER')
DataItem(text='fröhlicher gruß aus der schönsten stadt der welt theo ⚓️', coarse_label='OTHER', fine_label='OTHER')
DataItem(text='amis hätten alles und jeden gewählt...nur hillary wollten sie nicht und eine fortsetzung von obama-politik erst recht nicht..!', coarse_label='OTHER', fine_label='OTHER')
DataItem(text='kein verläßlicher verhandlungspartner. nachkarteln nach den sondierzngsgesprächen - schickt diese stümper spd in die versenkung.', coarse_label='OFFENSE', fine_label='INSULT')
DataItem(text='ja, aber wo widersprechen die zahlen denn denen, die im von uns verlinkten artikel stehen? in unserem tweet geht es rein um subs. geschützte. 2017 ist der gesamte familiennachzug im vergleich zu 2016 - die zahlen, die hr. brandner bemüht - übrigens leicht rückläufig gewesen.', coarse_label='OTHER', f

### Aufgabe 1.4: Training eines Naive-Bayes-Klassifikators

Mithilfe von `scikit-learn` ist es sehr einfach, eine Naive-Bayes-Klassifikator zu trainieren. Sie benötigen im einfachsten Fall nur den
[`CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) zur Vektorisierung und 
[`MultinomialNB`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html) für die Klassifikation zu einer Pipeline zu verbinden.

> 💡 **Tipp:** 
> Auch hier kann ChatGPT Ihnen beim Code helfen ...

Trainieren Sie den Klassifikator auf den Trainingsdaten und messen Sie die Accuracy auf den Testdaten. Wie gut ist Ihr Ergebnis? Vergleichen Sie Ihr Ergebnis mit den [Ergebnissen des GermEval-2018](../data/GermEval-2018/results.pdf).

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Nehmen wir an, `data_items` ist Ihre Liste von DataItem namedtuples
texts = [item.text for item in data_items]
labels = [item.coarse_label for item in data_items]

# Teilen der Daten in Training- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Erstellen der Pipeline
pipeline = make_pipeline(CountVectorizer(), MultinomialNB())

# Training des Klassifikators
pipeline.fit(X_train, y_train)

# Vorhersagen machen
y_pred = pipeline.predict(X_test)

# Genauigkeit bewerten
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.781437125748503


In [5]:
from sklearn.metrics import f1_score

# F1-Score berechnen
f1 = f1_score(y_test, y_pred, average='macro')  # 'macro' gibt an, dass der Durchschnitt über die Labels ungewichtet genommen wird.

print(f"Macro Average F1 Score: {f1}")

Macro Average F1 Score: 0.7152799673030887


In [6]:
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, f1_score

# Pfad zu den Dateien
train_file_path = '../data/GermEval-2018/germeval2018.training.txt'
test_file_path = '../data/GermEval-2018/germeval2018.test.txt'

def load_data(file_path):
    texts, labels = [], []
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter='\t')
        for row in reader:
            if row:  # Sicherstellen, dass die Zeile nicht leer ist
                texts.append(row[0])
                labels.append(row[1])  # Möglicherweise müssen Sie dies an Ihre spezifische Label-Spalte anpassen
    return texts, labels

# Laden der Trainings- und Testdaten
X_train, y_train = load_data(train_file_path)
X_test, y_test = load_data(test_file_path)

# Pipeline erstellen und Training
pipeline = make_pipeline(CountVectorizer(), MultinomialNB())
pipeline.fit(X_train, y_train)

# Vorhersage und Bewertung
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')

print(f"Accuracy: {accuracy}")
print(f"Macro Average F1 Score: {f1}")

Accuracy: 0.7292525014714538
Macro Average F1 Score: 0.6516265288756292


2